# 问题列表

## 常用图像标注工具

- [labelimg](https://github.com/tzutalin/labelImg)
- [labelme](https://github.com/wkentaro/labelme)
- [RectLable](https://rectlabel.com)
- [OpenCV/CVAT](https://github.com/cvat)
- [VOTT](https://github.com/microsoft/VoTT)
- [LabelBox](https://github.com/Labelbox/Labelbox)
- [VIA-VGG](http://www.robots.xo.ac.uk/~vgg/software/via)
- [PixelAnnotationTool](https://github.com/abreheret/PixelAnnotationTool)
- [point-cloud-annotation-tool](https://github.com/springzfx/point-clout-annotation-tool)
- [Boobs](https://github.com/drainingsum/boobs)

## labelme 数据标注 转换为VOC格式

```shell
# It generates:
#   - data_dataset_voc/JPEGImages
#   - data_dataset_voc/Annotations
#   - data_dataset_voc/AnnotationsVisualization
./labelme2voc.py data_annotated data_dataset_voc --labels labels.txt
```

## FFmpeg python 视屏抽帧

In [ ]:
import ffmpeg
import numpy as np
import os
import cv2


def get_video_shape(videopath):
    probe = ffmpeg.probe(videopath)
    video_stream = next(
        (stream
         for stream in probe['streams'] if stream['codec_type'] == 'video'),
        None)
    width = int(video_stream['width'])
    height = int(video_stream['height'])

    return width, height


def extract_frames_from_video(videopath, outdir=".", fps=10):
    width, height = get_video_shape(videopath)
    process = ffmpeg.input(videopath).filter(
        "fps", fps=fps,
        round="up").output('pipe:', format='rawvideo',
                           pix_fmt='rgb24').run_async(pipe_stdout=True)

    frame_id = 0
    while True:
        in_bytes = process.stdout.read(width * height * 3)
        if not in_bytes:
            break
        frame = (np.frombuffer(in_bytes, np.uint8).reshape([height, width, 3]))
        frame = cv2.cvtColor(frame, cv2.COLOR_RGB2BGR)

        cv2.imwrite(
            os.path.join(outdir,
                         os.path.splitext(os.path.split(videopath)[-1])[0]) +
            f"_{frame_id}.jpg", frame)
        frame_id += 1
    process.wait()


data_dirs = [
    "/opt/data/dataset/smoking/kinetics400smoking",
    "/opt/data/dataset/smoking/hmdb51-smoking/test/smoking",
    "/opt/data/dataset/smoking/hmdb51-smoking/train/smoking"
]

dst_dir = "/opt/data/dataset/smoking/extracts"

videofiles = [] 
for dir in data_dirs:
    videofiles += [
        os.path.join(dir, filename) for filename in next(os.walk(dir))[-1]
    ]

for videopath in videofiles:
    print(f"extract frame video :  {videopath}")
    extract_frames_from_video(videopath, outdir=dst_dir, fps=0.25)

## FFMPEG 编译安装

1.  配置(其中GPU支持需要安装nv-codec-headers)
```bash
# 默认CPU版本
./configure --enable-shared --disable-ffplay --disable-doc --disable-x86asm
```

`nv-codec-headers`安装
```bash
git clone https://github.com/FFmpeg/nv-codec-headers.git
cd nv-codec-headers
make && make install
```

```bash
# GPU支持
./configure --enable-shared --disable-ffplay --disable-doc --enable-cuda-nvcc --enable-nonfree --disable-x86asm --enable-avresample --enable-cuvid --enable-nvdec --enable-nvenc
```

2. 安装
```bash
make && make install
```

## libSM.so.6

### 错误描述

opencv-python 在`import cv2`的时候出现

```bash
Traceback (most recent call last):
  File "main_server.py", line 4, in <module>
    import cv2
  File "/usr/local/lib/python3.6/dist-packages/cv2/__init__.py", line 3, in <module>
    from .cv2 import *
ImportError: libSM.so.6: cannot open shared object file: No such file or directory
```

### 解决办法：


```bash
apt install libsm-dev
```

##  libXrender.so.1

### 错误信息：
opencv-python 在`import cv2`的时候出现

```bash
Traceback (most recent call last):
  File "main_server.py", line 4, in <module>
    import cv2
  File "/usr/local/lib/python3.6/dist-packages/cv2/__init__.py", line 3, in <module>
    from .cv2 import *
ImportError: libXrender.so.1: cannot open shared object file: No such file or directory
```

### 解决办法：

```bash
apt install libxrender-dev
```

## Docker 

1. 删除所有无用镜像：

```bash
sudo docker image rm $(sudo docker images | grep "<none>" | awk '{print $3}')
```

2. 删除所有容器：

```bash
sudo docker rm $(sudo docker ps -aq)
```

3. 提交镜像：

```bash
sudo docker commit 9b91f151eb40 ccr.ccs.tencentyun.com/zyuegege/testimage:mydev
```

## OpenCV 编译(支持CUDA加速)
1. 执行`cmake`
```bash
cd opencv-4.1.2/build
cmake \
       -DCMAKE_BUILD_TYPE=Release \
       -DOPENCV_EXTRA_MODULES_PATH=../../opencv_contrib/modules \
       -DPYTHON3_EXECUTABLE=/usr/bin/python3.6m \
       -DPYTHON_INCLUDE_DIR=/usr/include/python3.6m \
       -DPYTHON_INCLUDE_DIR2=/usr/include/x86_64-linux-gnu/python3.6m \
       -DPYTHON_LIBRARY=/usr/lib/x86_64-linux-gnu/libpython3.6m.so \
       -DOPENCV_GENERATE_PKGCONFIG=ON \
       -DBUILD_opencv_python3=ON \
       -DBUILD_opencv_python2=OFF \
       -DBUILD_JAVA=OFF \
       -DWITH_CUDA=ON \
       -DWITH_FFMPEG=ON  \
       -DPYTHON3_NUMPY_INCLUDE_DIRS=/usr/include/python3.6/numpy
       ..
```

## jupyter lab 

### 问题 `KeyError: 'registry'`:

```bash
Traceback (most recent call last):
  File "/usr/local/lib/python3.6/dist-packages/traitlets/traitlets.py", line 528, in get
    value = obj._trait_values[self.name]
KeyError: 'registry'

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/usr/local/bin/jupyter-lab", line 5, in <module>
    from jupyterlab.labapp import main
  File "/usr/local/lib/python3.6/dist-packages/jupyterlab/labapp.py", line 40, in <module>
    app_version = get_app_version()
  File "/usr/local/lib/python3.6/dist-packages/jupyterlab/commands.py", line 541, in get_app_version
    app_options, app_dir=app_dir, core_config=core_config))
  File "/usr/local/lib/python3.6/dist-packages/jupyterlab/commands.py", line 582, in __init__
    self.registry = options.registry
  File "/usr/local/lib/python3.6/dist-packages/traitlets/traitlets.py", line 556, in __get__
    return self.get(obj, cls)
  File "/usr/local/lib/python3.6/dist-packages/traitlets/traitlets.py", line 535, in get
    value = self._validate(obj, dynamic_default())
  File "/usr/local/lib/python3.6/dist-packages/jupyterlab/commands.py", line 343, in _default_registry
    config = _yarn_config(self.logger)["yarn config"]
  File "/usr/local/lib/python3.6/dist-packages/jupyterlab/commands.py", line 1765, in _yarn_config
    node = which('node')
  File "/usr/local/lib/python3.6/dist-packages/jupyterlab_server/process.py", line 59, in which
    raise ValueError(msg)
ValueError: Please install nodejs 5+ and npm before continuing installation. nodejs may be installed using conda or directly from the nodejs website.
```

### 解决办法:

```bash
pip install jupyterlab==1.1.3
```

## Ubuntu 设置时区

```bash
tzselect
sudo cp /usr/share/zoneinfo/Asia/Shanghai /etc/localtime
```

## Docker 共享内内存区大小

一般默认情况Linux系统会挂在一半大小的物理内存用与共享内存
```bash
df -h

/dev/nvme0n1p2  234G   58G  165G   26% /
tmpfs            16G  104M   16G    1% /dev/shm
```

但是默认的Docker的共享内存去大小只有64M
```bash
df -h 

shm              64M     0   64M 0% /dev/shm
```

通过在运行docker时加上`--shm-size=16G`可以改变其大小，例如：
```bash
docker run -d --gpus all  -v /opt:/opt -v /opt/git/alnotes:/var/notebook --shm-size=16G zyuegege/devel:10.0-cudnn7-devel-ubuntu18.04
```




## 查看IPC共享内存区限制

参考：[【Linux - Shared Memory (SHM) (/dev/shm)】](https://gerardnico.com/os/linux/shared_memory)

1. IPC共享内存的文件系统`/dev/shm`，使用`df  -k -h /dev/shm`查看其大小；
2. 限制
```bash
ipcs -lm --human
```

```text
---------- 同享内存限制 ------------  
最大段数 = 4096  
最大段大小 = 16E  
最大总共享内存 = 16E  
最小段大小 = 1B
```

3. 参数
- `shmseg - max` # of shared memory segs per process: (doesn't exist in FC6)
- `shmmni - max` # of shared memory segs system-wide
```bash
cat /proc/sys/kernel/shmmax
cat /proc/sys/kernel/shmall
cat /proc/sys/kernel/shmmni
```

4. 查看适用情况
```bash
# 查看共享内存信息
ipcs -m
# 查看共享内存修改情况
ipcs -pm
```


## c/c++ 利用Python API调用Python脚本出现threading的错误

```bash
```

解决办法：利用[`boost::python`](https://www.boost.org/doc/libs/1_71_0/libs/python/doc/html/index.html)库代替

## pip 安装 `No module named 'distutils.util'`

### 错误信息：

```bash
Traceback (most recent call last):
  File "get-pip.py", line 22318, in <module>
    main()
  File "get-pip.py", line 198, in main
    bootstrap(tmpdir=tmpdir)
  File "get-pip.py", line 82, in bootstrap
    import pip._internal.main
  File "/tmp/tmptdvv_4w8/pip.zip/pip/_internal/main.py", line 13, in <module>
  File "/tmp/tmptdvv_4w8/pip.zip/pip/_internal/cli/autocompletion.py", line 11, in <module>
  File "/tmp/tmptdvv_4w8/pip.zip/pip/_internal/cli/main_parser.py", line 7, in <module>
  File "/tmp/tmptdvv_4w8/pip.zip/pip/_internal/cli/cmdoptions.py", line 19, in <module>
ModuleNotFoundError: No module named 'distutils.util'
```

### 解决办法：
```bash
apt-get install python3-distutils
```

## pip 国内源

1. pip国内源地址：

- 清华：[https://pypi.tuna.tsinghua.edu.cn/simple](https://pypi.tuna.tsinghua.edu.cn/simple)
- 阿里云：[http://mirrors.aliyun.com/pypi/simple](http://mirrors.aliyun.com/pypi/simple)
- 中国科技大学： [https://pypi.mirrors.ustc.edu.cn/simple](https://pypi.mirrors.ustc.edu.cn/simple)
- 华中理工大学：[http://pypi.hustunique.com/](http://pypi.hustunique.com/)
- 山东理工大学：[http://pypi.sdutlinux.org/](http://pypi.sdutlinux.org/) 
- 豆瓣：[http://pypi.douban.com/simple/](http://pypi.douban.com/simple/)

2. 适用方法：
- 临时适用：

```bash
pip install -i http:pypi.douban.com/simple/ django 
```

- pip 使用非 https 的镜像地址后无法使用，提示每次要加 `--trusted-host url` 来解决

```bash
pip install django -i http://pypi.douban.com/simple --trusted-host pypi.douban.com
```

- 永久适用：

    修改配置文件`~/.pip/pip.conf`的内容为下：

```bash
[global]
index-url = http://mirrors.aliyun.com/pypi/simple
[install]
trusted-host=mirrors.aliyun.com
```

## pip 安装`pycurl` 出现`No such file or directory: 'curl-config': 'curl-config'`

### 具体错误：

```bash
ERROR: Command errored out with exit status 1:
     command: /usr/bin/python -c 'import sys, setuptools, tokenize; sys.argv[0] = '"'"'/tmp/pip-install-9_iiqsx6/pycurl/setup.py'"'"'; __file__='"'"'/tmp/pip-install-9_iiqsx6/pycurl/setup.py'"'"';f=getattr(tokenize, '"'"'open'"'"', open)(__file__);code=f.read().replace('"'"'\r\n'"'"', '"'"'\n'"'"');f.close();exec(compile(code, __file__, '"'"'exec'"'"'))' egg_info --egg-base /tmp/pip-install-9_iiqsx6/pycurl/pip-egg-info
         cwd: /tmp/pip-install-9_iiqsx6/pycurl/
    Complete output (22 lines):
    Traceback (most recent call last):
      File "/tmp/pip-install-9_iiqsx6/pycurl/setup.py", line 229, in configure_unix
        stdout=subprocess.PIPE, stderr=subprocess.PIPE)
      File "/usr/lib/python3.6/subprocess.py", line 729, in __init__
        restore_signals, start_new_session)
      File "/usr/lib/python3.6/subprocess.py", line 1364, in _execute_child
        raise child_exception_type(errno_num, err_msg, err_filename)
    FileNotFoundError: [Errno 2] No such file or directory: 'curl-config': 'curl-config'
    
    During handling of the above exception, another exception occurred:
    
    Traceback (most recent call last):
      File "<string>", line 1, in <module>
      File "/tmp/pip-install-9_iiqsx6/pycurl/setup.py", line 944, in <module>
        ext = get_extension(sys.argv, split_extension_source=split_extension_source)
      File "/tmp/pip-install-9_iiqsx6/pycurl/setup.py", line 606, in get_extension
        ext_config = ExtensionConfiguration(argv)
      File "/tmp/pip-install-9_iiqsx6/pycurl/setup.py", line 101, in __init__
        self.configure()
      File "/tmp/pip-install-9_iiqsx6/pycurl/setup.py", line 233, in configure_unix
        raise ConfigurationError(msg)
    __main__.ConfigurationError: Could not run curl-config: [Errno 2] No such file or directory: 'curl-config': 'curl-config'
    ----------------------------------------
ERROR: Command errored out with exit status 1: python setup.py egg_info Check the logs for full command output.
```

### 解决办法：

```bash
apt install curl libcurl4-openssl-dev
```

## pip `'x86_64-linux-gnu-gcc'`

### 错误信息：

```bash
    ERROR: Command errored out with exit status 1:
     command: /usr/bin/python -u -c 'import sys, setuptools, tokenize; sys.argv[0] = '"'"'/tmp/pip-install-i93ggql9/pycurl/setup.py'"'"'; __file__='"'"'/tmp/pip-install-i93ggql9/pycurl/setup.py'"'"';f=getattr(tokenize, '"'"'open'"'"', open)(__file__);code=f.read().replace('"'"'\r\n'"'"', '"'"'\n'"'"');f.close();exec(compile(code, __file__, '"'"'exec'"'"'))' install --record /tmp/pip-record-isilpo9p/install-record.txt --single-version-externally-managed --compile
         cwd: /tmp/pip-install-i93ggql9/pycurl/
    Complete output (16 lines):
    Using curl-config (libcurl 7.58.0)
    Using SSL library: OpenSSL/LibreSSL/BoringSSL
    running install
    running build
    running build_py
    creating build
    creating build/lib.linux-x86_64-3.6
    creating build/lib.linux-x86_64-3.6/curl
    copying python/curl/__init__.py -> build/lib.linux-x86_64-3.6/curl
    running build_ext
    building 'pycurl' extension
    creating build/temp.linux-x86_64-3.6
    creating build/temp.linux-x86_64-3.6/src
    x86_64-linux-gnu-gcc -pthread -DNDEBUG -g -fwrapv -O2 -Wall -g -fstack-protector-strong -Wformat -Werror=format-security -Wdate-time -D_FORTIFY_SOURCE=2 -fPIC -DPYCURL_VERSION="7.43.0.3" -DHAVE_CURL_SSL=1 -DHAVE_CURL_OPENSSL=1 -DHAVE_CURL_SSL=1 -I/usr/include/python3.6m -c src/docstrings.c -o build/temp.linux-x86_64-3.6/src/docstrings.o
    unable to execute 'x86_64-linux-gnu-gcc': No such file or directory
    error: command 'x86_64-linux-gnu-gcc' failed with exit status 1
    ----------------------------------------
ERROR: Command errored out with exit status 1: /usr/bin/python -u -c 'import sys, setuptools, tokenize; sys.argv[0] = '"'"'/tmp/pip-install-i93ggql9/pycurl/setup.py'"'"'; __file__='"'"'/tmp/pip-install-i93ggql9/pycurl/setup.py'"'"';f=getattr(tokenize, '"'"'open'"'"', open)(__file__);code=f.read().replace('"'"'\r\n'"'"', '"'"'\n'"'"');f.close();exec(compile(code, __file__, '"'"'exec'"'"'))' install --record /tmp/pip-record-isilpo9p/install-record.txt --single-version-externally-managed --compile Check the logs for full command output.
```

### 解决办法：

```bash
apt install gcc make cmake g++
```

## `Python.h`

### 错误描述

```bash
fatal error: Python.h: No such file or directory
#include <Python.h>
```


### 解决办法

```bash
apt install libpython3-dev
```

## `openssl/ssl.h`

### 错误描述：

```bash
fatal error: openssl/ssl.h: No such file or directory
#include <openssl/ssl.h>
```

### 解决办法：

```bash
apt install libssl-dev
```